In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
# pd.set_option('max_columns', None)  # show all columns
from sklearn.preprocessing import OneHotEncoder
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

In [2]:
df = pd.read_excel('../BJK_corregido.xlsx')

In [108]:
df_2 = pd.read_excel('../BJK_corregido.xlsx')  # to make changes

### Prepare data:
Discrete vars:  
- ```[setNumber, gameNumber, pointNumber, serveNumber, shotCount, efectividad,
1º o 2º saque, seve_class, Lado(1:iguales, 0:ventaja), DIRECCIÓN, X1,...]```

Discrete vars to delete (only one value) --> no information!  
```[Point,```

Continuous vars:  
- ```v(m/s), v(km/h), timeHIT,Y1, Y1(ABS), Z1(h), timeNET, TimeN-TimeH,
timeBounce, TimeB-TimeH,PreVx, PreVy, PreVz, PreV, PostVx, PostVy,
PostVz, PostV, Dif. V1-V2, Dif(km/h)```


Vars to delete (duplicated or all the same):
- ```v(km/h), Dif. (km/h), Point,```

In [132]:
### Check how many rows have empty values or NaNs
print(df_2.isnull().sum().sum(), '/', len(df_2))  ### too many, we can't just delete rows with empty values --> interpolate

17498 / 29193


In [133]:
### interpolate NaN with none-linearities
for i in df_2.columns:
    df_2[i] = df_2[i].interpolate(method='polynomial', order=2)

In [143]:
print(df_2.isnull().sum().sum(), '/', len(df_2))  ### too many, we can't just delete rows with empty values --> interpolate

0 / 29193


In [4]:
# sorted(df_2.columns)

startScore_Server  = [0 15 30 40 50 3 4 6 1 5 7 8 2 'WIN' 9 10 11 12 13]  
startScore_Receiver = [0 15 40 50 30 1 2 5 6 3 7 4 8 'LOS' 9 10 11 12 13]

I don't understand this variable ... not in a format the machine can understand. Delete for the moment

EventID: I don't think the machine can understand this variable for the moment ...   
   not clear how to show to the machine (162 events?)... we will show all events as one for the moment

In [109]:
### delete vars
df_2.drop(['V (km/h)', 'Dif. (km/h)', 'Point', 'EventID', 'startScore_Server', 'startScore_Receiver'], axis=1, inplace=True)

In [110]:
### Ordinal vars can usually be kept sa they are, since we assume that their order is the right order
discrete_ordinal = ["setNumber", 
                 "gameNumber", 
                 "pointNumber", 
                 "serveNumber", 
                 "shotCount", 
                 "Efectividad",
                 "1º o 2º saque", 
                 "Lado(1:Iguales;0:Ventaja)", 
                 "DIRECCIÓN:1 abierto;2 al cuerpo;3 a la T"]

### discrete one_hot encoding are vars that are strings or not ordinal, so they need to be unfolded into binary vars
discrete_one_hot = ["serve_class"]

discrete_one_hot

['serve_class']

In [111]:
def one_hot(df, cols_list):
    """
    cols_list=['service', 'gender', ...]
    """
    #perform one-hot encoding on 'team' column
    for i in cols_list:
        print(f"one hot on '{i}'")
        df_discrete = df[cols_list]
        encoder_df = pd.DataFrame(encoder.fit_transform(df_discrete[[i]]).toarray())
        ### rename columns
        print('encoder.categories_: ', encoder.categories_[0].tolist())
        l = [i+'___'+str(x) for x in encoder.categories_[0]]
        # print(l)
        encoder_df.columns = l # encoder.categories_
        # merge one-hot encoded columns back with original DataFrame
        df = pd.concat([df, encoder_df], axis=1)
    df.drop(cols_list, axis=1, inplace=True)
    return df

In [134]:
df_3 = one_hot(df_2, discrete_one_hot)

one hot on 'serve_class'
encoder.categories_:  ['ACE', 'FAULT', 'NOT RETURNED', 'RETURNED']


In [135]:
df_3

,Sexo,setNumber,gameNumber,pointNumber,serveNumber,shotCount,Efectividad,1º o 2º saque,Lado(1:Iguales;0:Ventaja),DIRECCIÓN:1 abierto;2 al cuerpo;3 a la T,...,PreV,PostVx,PostVy,PostVz,PostV,Dif. V1-V2,serve_class___ACE,serve_class___FAULT,serve_class___NOT RETURNED,serve_class___RETURNED
0,1,1,1,1,1,2,1,1,1,2,...,31.203651,21.263,5.684,6.380,22.915659,8.287992,0.0,0.0,1.0,0.0
1,1,1,1,2,1,4,0,1,0,1,...,35.767428,24.944,-5.850,6.801,26.508098,9.259331,0.0,0.0,0.0,1.0
2,1,1,1,3,1,2,1,1,1,2,...,30.302482,21.009,5.585,7.058,22.855758,7.446724,0.0,0.0,1.0,0.0
3,1,1,1,4,1,1,1,2,0,1,...,24.999723,15.163,-3.431,6.906,17.011207,7.988516,0.0,1.0,0.0,0.0
4,1,1,2,1,1,4,0,1,1,2,...,27.657937,18.158,2.731,6.956,19.635612,8.022325,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29188,1,3,9,1,1,2,0,1,1,3,...,33.999023,-23.728,-2.379,6.608,24.745571,9.253452,0.0,0.0,1.0,0.0
29189,1,3,9,3,1,1,0,2,0,2,...,29.762144,-19.062,-4.510,7.014,20.806156,8.955988,0.0,1.0,0.0,0.0
29190,1,3,9,4,1,3,0,1,0,1,...,27.476137,-17.347,6.608,6.992,19.836132,7.640005,0.0,0.0,0.0,1.0
29191,1,3,9,5,1,1,0,2,0,3,...,31.018599,-21.329,-2.954,6.647,22.535194,8.483405,0.0,1.0,0.0,0.0


In [136]:
corr = df_3.corr(method='pearson').Efectividad.sort_values(ascending=False)
correlated_cols = list(corr[corr>0].index)

In [137]:
correlated_cols

['Efectividad',
 'serve_class___NOT RETURNED',
 'serve_class___ACE',
 'PreV',
 'V (m/s)',
 'Y1(ABS)',
 'Dif. V1-V2',
 'Z1 (h)',
 'setNumber',
 'X1',
 'PostV',
 'Lado(1:Iguales;0:Ventaja)',
 'serveNumber',
 'Y1']

In [138]:
corr

Efectividad                                 1.000000
serve_class___NOT RETURNED                  0.588071
serve_class___ACE                           0.298286
PreV                                        0.196316
V (m/s)                                     0.192850
Y1(ABS)                                     0.096717
Dif. V1-V2                                  0.094357
Z1 (h)                                      0.061928
setNumber                                   0.031322
X1                                          0.014339
PostV                                       0.013231
Lado(1:Iguales;0:Ventaja)                   0.013065
serveNumber                                 0.008910
Y1                                          0.000038
PreVy                                      -0.000053
PostVy                                     -0.000458
gameNumber                                 -0.001562
PostVx                                     -0.015682
PreVx                                      -0.

#### **Conclusión:**
las correlaciones positivas son las que están correlacionadas con efectividad = 1,
pero las negativas pueden estar correlacionadas con efectividad nula, la cual es info válida tb.

Vamos a preparar dos datasets, uno con las positivas y otro con todas, Entrenar un par de modelos y sacar conclusiones.

Also, for the moment we are goind to delete the eventID --> 162 different events ... 

In [139]:
df_positive_corr = df_3[correlated_cols]
df_positive_corr

,Efectividad,serve_class___NOT RETURNED,serve_class___ACE,PreV,V (m/s),Y1(ABS),Dif. V1-V2,Z1 (h),setNumber,X1,PostV,Lado(1:Iguales;0:Ventaja),serveNumber,Y1
0,1,1.0,0.0,31.203651,41.47,2.286,8.287992,3.174,1,-11.389,22.915659,1,1,-2.286
1,0,0.0,0.0,35.767428,50.91,0.871,9.259331,2.886,1,-11.385,26.508098,0,1,0.871
2,1,1.0,0.0,30.302482,41.87,1.964,7.446724,2.912,1,-11.387,22.855758,1,1,-1.964
3,1,0.0,0.0,24.999723,35.22,0.817,7.988516,3.030,1,-11.384,17.011207,0,1,0.817
4,0,0.0,0.0,27.657937,39.27,1.088,8.022325,2.904,1,-11.390,19.635612,1,1,-1.088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29188,0,1.0,0.0,33.999023,47.45,1.007,9.253452,2.719,3,11.385,24.745571,1,1,1.007
29189,0,0.0,0.0,29.762144,41.82,1.427,8.955988,2.703,3,11.379,20.806156,0,1,1.427
29190,0,0.0,0.0,27.476137,38.41,2.411,7.640005,2.956,3,11.385,19.836132,0,1,-2.411
29191,0,0.0,0.0,31.018599,44.69,1.232,8.483405,2.729,3,11.388,22.535194,0,1,1.232


In [148]:
# Code source: Gael Varoquaux   # License: BSD 3 clause
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

def logistic_and_linear(df, target_var='Efectividad'):
    y = df[target_var].values.reshape(-1, 1)
    X = df.drop(columns=[target_var]).values
    print(f"check shapes. X: {X.shape}, y: {y.shape}")
    # X, y = df['V (m/s)'].values.reshape(-1, 1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
    # Logistic:
    clf = LogisticRegression(penalty='l2') # dual=True, solver='liblinear')
    clf.fit(X, y)
    plt.figure(1, figsize=(12, 8))
    plt.clf()
    plt.scatter(X.ravel(), y, color="black")
    loss = expit(X_test * clf.coef_ + clf.intercept_).ravel()
    plt.plot(X_test, loss, color="red", linewidth=1)
    ### OLS:
    ols = LinearRegression()
    ols.fit(X, y)
    plt.plot(X_test, ols.coef_ * X_test + ols.intercept_, linewidth=1)
    ### plt ...
    plt.ylabel("y")
    plt.xlabel("X")
    plt.legend(
        ("Logistic Regression Model", "Linear Regression Model"),
        loc="lower right",
        fontsize="small",
    )
    plt.tight_layout()
    plt.show()

In [141]:
df_3.dtypes

Sexo                                          int64
setNumber                                     int64
gameNumber                                    int64
pointNumber                                   int64
serveNumber                                   int64
shotCount                                     int64
Efectividad                                   int64
1º o 2º saque                                 int64
Lado(1:Iguales;0:Ventaja)                     int64
DIRECCIÓN:1 abierto;2 al cuerpo;3 a la T      int64
V (m/s)                                     float64
timeHIT                                     float64
X1                                          float64
Y1                                          float64
Y1(ABS)                                     float64
Z1 (h)                                      float64
timeNET                                     float64
TimeN-TimeH                                 float64
timeBounce                                  float64
TimeB-TimeH 

In [142]:
for i in df_3.columns:
    print(i, df_3[i].unique())

Sexo [1 2]
setNumber [1 2 3 4 5]
gameNumber [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46]
pointNumber [ 1  2  3  4  6  7  8  9 10  5 11 12 14 13 15 16 17 18 19 20 21 22 23 24
 25 26 27]
serveNumber [1 2]
shotCount [ 2  4  1  8  3  7  9  5  6 10 12 13 15 11 18 16 17 21 14 20 19 22 25 26
 23 24]
Efectividad [1 0]
1º o 2º saque [1 2]
Lado(1:Iguales;0:Ventaja) [1 0]
DIRECCIÓN:1 abierto;2 al cuerpo;3 a la T [2 1 3]
V (m/s) [41.47 50.91 41.87 ... 32.78 32.14 58.96]
timeHIT [0.441 0.128 4.312 ... 2.373 2.301 3.765]
X1 [-11.389 -11.385 -11.387 ...  11.022  11.686  11.737]
Y1 [-2.286  0.871 -1.964 ... -3.898 -3.836 -3.948]
Y1(ABS) [2.286      0.871      1.964      ... 3.06203507 2.80759489 2.5495832 ]
Z1 (h) [3.174      2.886      2.912      ... 2.79755844 2.80366876 2.80981595]
timeNET [0.754 0.387 4.622 ... 2.884 2.741 4.032]
TimeN-TimeH [0.313 0.259 0.31  ... 0.397 0.426 0.443]
timeBounce [0.915 0.

In [155]:
# https://stackabuse.com/implementing-svm-and-kernel-svm-with-pythons-scikit-learn/
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X, y = df_3.values, df_3.Efectividad.values # .reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

svclassifier = SVC(kernel='rbf')
svclassifier.fit(X_train, y_train)

y_pred = svclassifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[4202    0]
 [1637    0]]
              precision    recall  f1-score   support

           0       0.72      1.00      0.84      4202
           1       0.00      0.00      0.00      1637

    accuracy                           0.72      5839
   macro avg       0.36      0.50      0.42      5839
weighted avg       0.52      0.72      0.60      5839



/mnt/c/Users/JLazaro/notebook/jupyterenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/c/Users/JLazaro/notebook/jupyterenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/c/Users/JLazaro/notebook/jupyterenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

In [158]:
X, y = df_positive_corr.values, df_positive_corr.Efectividad.values # .reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

svclassifier = SVC(kernel='rbf')
svclassifier.fit(X_train, y_train)

y_pred = svclassifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[4061    0]
 [1778    0]]
              precision    recall  f1-score   support

           0       0.70      1.00      0.82      4061
           1       0.00      0.00      0.00      1778

    accuracy                           0.70      5839
   macro avg       0.35      0.50      0.41      5839
weighted avg       0.48      0.70      0.57      5839



/mnt/c/Users/JLazaro/notebook/jupyterenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/c/Users/JLazaro/notebook/jupyterenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/mnt/c/Users/JLazaro/notebook/jupyterenv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

In [157]:
df_3.Efectividad.values.shape,  df_3.Efectividad.values.reshape(-1,1).shape

((29193,), (29193, 1))

## Try all possible algorithms and compare! 
Here starts the fun part 

## Lessons learnt - TODOES:  
- use df_3, not the top correlated df  
- using V vs Efectividad gives almos tthe same results!  
- Try all algorithms with all 3 X options!!  
    - V vs Efectividad
    - df_3 vs Efectividad
    - Max_correlated_vars_df vs Efectividad